<a href="https://colab.research.google.com/github/ak-hannou/compsci-4ml3/blob/main/F24_4ML3_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4: Application of Neural Networks for Image Processing

In this assignment, you will complete three main tasks to deepen your understanding of neural networks, specifically using Convolutional Neural Networks (CNNs) for image processing:

1. **Image Classification with CNN (30 points)**  
   Create an image classifier using PyTorch, building and training a CNN model to classify images accurately.

2. **Image Denoising with CNN (45 points)**  
   Develop a denoising model using PyTorch, leveraging a CNN to remove noise from images and enhance visual quality.

3. **Kaggle Competition (25 + 20 points)**  
   Participate in a Kaggle competition where you will apply your skills in image classification on a challenging dataset.

---

# Submission Guidelines

### Your submission on A2L should include **five files** in total:

1. **Report** (`report.pdf`): Document your results and answer all assignment-related questions in this PDF.

2. **Code** (`F24-4ML3-Assignment 4.ipynb`): Submit your code in a Jupyter Notebook file, maintaining the original notebook structure provided.

3. **Kaggle Code**: Provide a separate file with the code specifically used for the Kaggle competition.

4. **Kaggle Report** (`kaggle.pdf`): Write a concise report explaining your approach and model used in the competition.

5. **Submission File** (`submission.csv`): Refer to the Kaggle Competition section for details on generating and uploading this file to A2L.


---

**Total Points**: 120 (with an additional 20 bonus points available for Kaggle competition)

**Submission Deadline**: December 2, 2024 11:59PM


In [ ]:
# Importing necessary libraries

import torch  # Core PyTorch library for tensor operations
from torchvision import datasets, transforms  # Datasets and transformations for computer vision
import torch.nn as nn  # Neural network components
import torch.optim as optim  # Optimization algorithms
import numpy as np  # Numerical operations
import matplotlib.pyplot as plt  # Plotting
from torch.utils.data import DataLoader, Dataset, random_split, Subset  # Data handling utilities

# SVHN Dataset

In this assignment, you will work with the **Street View House Numbers (SVHN) dataset** to develop a denoising algorithm and a classification task. The SVHN dataset consists of over **600,000** color images of house numbers collected from Google Street View images. Each image is a 32x32 pixel RGB image capturing digits (0-9) in real-world scenarios. The dataset can be found at [this link](http://ufldl.stanford.edu/housenumbers/).

For this assignment, you will use:

- **Training set**: 73,257 images
- **Testing set**: 26,032 images

In [ ]:
# Define transformations for the dataset
transform = transforms.ToTensor()

# Load the training and test datasets
trainset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
testset = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

# Set batch size as a parameter
batch_size = 64

# Function to create data loaders with a specified batch size
def get_dataloader(dataset, batch_size, shuffle):
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

# Create data loaders for training and testing
trainloader = get_dataloader(trainset, batch_size=batch_size, shuffle=True)
testloader = get_dataloader(testset, batch_size=batch_size, shuffle=False)

# Visualization of SVHN dataset

In [ ]:
# Define the class labels for the SVHN dataset
classes = [str(i) for i in range(10)]

# Function to display a grid of images with specified rows, columns, and class labels
def show_images_grid(dataloader, rows, cols):
    # Get a batch of training images and labels
    dataiter = iter(dataloader)
    images, labels = next(dataiter)

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))
    idx = 0
    for i in range(rows):
        for j in range(cols):
            if idx < len(images):
                npimg = images[idx].numpy()
                axes[i, j].imshow(np.transpose(npimg, (1, 2, 0)))
                axes[i, j].set_title(f'Class: {classes[labels[idx]]}')
                axes[i, j].axis('off')
                idx += 1
    plt.tight_layout()
    plt.show()

In [ ]:
# Display a 2x5 grid of images from trainloader
show_images_grid(trainloader, rows=2, cols=5)

The `del` keyword deletes variables from memory, freeing up space. This is useful in large projects, especially deep learning, where datasets and data loaders use significant memory. Removing these variables ensures available memory for later code and helps avoid conflicts with similar names.


In [ ]:
del trainset, testset, trainloader, testloader, transform, batch_size, get_dataloader

# <font color="red">Task 1: SVHN Image Classification Using a Convolutional Neural Network (CNN) (30 points) </font>

In this task, you will complete the implementation of a CNN model and use it to classify images from the SVHN dataset. Your goal is to build an accurate model that can recognize and classify house number digits in real-world images.


When available, a GPU can accelerate neural network training with parallel computations, ideal for large datasets and complex models.


In [ ]:
# Set up device for training (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Hyper-Parameters:

In [ ]:
batch_size = 64           # Batch size for data loading
learning_rate = 0.05      # Learning rate for optimizer
num_epochs = 20           # Number of training epochs

Loading the dataset

In [ ]:
# Define transformations for SVHN dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970))
])

# Load training and test datasets with transformations
train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

# Create data loaders with specified batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## **A (15 points)** Build the CNN Model


A convolutional layer can be implemented in pytorch like

```
nn.Conv2d(in_channels=, out_channels=, kernel_size=, padding=, stride=).
```

Complete the CNN model implementation with the following architecture:

1. **Convolutional Layers**:
   - Three convolutional layers with:
     - `kernel_size=3`, `stride=1`, `padding=1`
     - Output channels:
       - First layer: 32 channels
       - Second layer: 64 channels
       - Third layer: 128 channels
   - Each convolutional layer is followed by:
     - Batch normalization to stabilize and speed up training
     - ReLU activation for non-linearity
     - Max pooling (`kernel_size=2`, `stride=2`) to reduce spatial dimensions

2. **Fully Connected Layers**:
   - Flatten the output from the convolutional layers.
   - First fully connected layer with 128 units and ReLU activation.
   - Final fully connected layer with 10 units for classification (one for each class in SVHN).

In [ ]:
class SVHNClassifier(nn.Module):
    def __init__(self):
        super(SVHNClassifier, self).__init__()

        ########################
        ########################
        #### YOUR CODE HERE ####
        ########################
        ########################


    def forward(self, x):

        ########################
        ########################
        #### YOUR CODE HERE ####
        ########################
        ########################


## **B (15 points)** Train the network
In the part, we are going to train the SVHN_CNN model to classify SVHN dataset.

1. Finish implementing train and test functions.
2. Include last epoch results for train/test loss and train/test accuracy in your report
3. Plot the train/test losses and train/test accuracies using the plot_metrics function.


In [ ]:
# Initialize model and move to device
model = SVHNClassifier().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # CrossEntropy includes softmax
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# Define function to evaluate model performance
def evaluate(model, data_loader, criterion):

    ########################
    ########################
    #### YOUR CODE HERE ####
    ########################
    ########################

    with torch.no_grad():  # No gradient calculation for evaluation
        ########################
        ########################
        #### YOUR CODE HERE ####
        ########################
        ########################



    ########################
    ########################
    #### YOUR CODE HERE ####
    ########################
    ########################


    return accuracy, avg_loss

In [ ]:
# Training the model
train_losses, test_losses = [], []
train_accuracies, test_accuracies = [], []

for epoch in range(num_epochs):

    ########################
    ########################
    #### YOUR CODE HERE ####
    ########################
    ########################

    # Evaluate on train and test sets after each epoch

    ########################
    ########################
    #### YOUR CODE HERE ####
    ########################
    ########################


    print(f'Epoch {epoch+1:02d}/{num_epochs:02d} - Train Loss: {train_loss:.6f}, Train Acc: {train_accuracy:.2f}%')
    print(f'            - Test Loss: {test_loss:.6f}, Test Acc: {test_accuracy:.2f}%')
    print("-" * 60)

    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

In [ ]:
def plot_metrics(train_metrics, test_metrics, metric_name):
    plt.figure(figsize=(8, 6))
    epochs = np.arange(len(train_metrics))

    plt.plot(epochs, train_metrics, label=f'Train {metric_name}', color='blue')
    plt.plot(epochs, test_metrics, label=f'Test {metric_name}', color='red')

    plt.xlabel('Epochs')
    plt.ylabel(metric_name)
    plt.title(f'{metric_name} over Epochs')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot loss
plot_metrics(train_losses, test_losses, 'Loss')

In [ ]:
# Plot accuracy
plot_metrics(train_accuracies, test_accuracies, 'Accuracy')

# <font color="red">Task 2: SVHN Denoising with Convolution Neural Network (CNN) (45 points) </font>
Convolutional Neural Network (CNN) have also been quite successful in the field of image processing. In this part, you are asked to finish the implementation of the CNN model and use the model to denoise images from SVHN.

The objective of tasks in the assignment is to train a network that, given a noisy image, recovers the original image. Therefore, each training point consists of the input (noisy image) and the expected output (true image).






Again, we remove these variables ensures available memory for later code and helps avoid conflicts with similar names.

In [ ]:
del device, batch_size, learning_rate, num_epochs,\
     train_dataset, test_dataset, train_loader, test_loader, transform,\
     SVHNClassifier, model, criterion, optimizer, evaluate,\
     train_losses, test_losses, train_accuracies, test_accuracies, show_images_grid, classes

In [ ]:
# Set up device for training (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Create the Noisy dataset

### Step 1: Defining a Custom Noise Transformation
First, we define a custom transformation that applies noise to each image by randomly dropping pixels. This transformation will be used to generate a noisy version of every image in the dataset.

In [ ]:
# Custom transformation to add noise by dropping pixels with a given probability for each channel independently
class AddNoiseTransform:
    def __init__(self, drop_prob=0.1):
        self.drop_prob = drop_prob

    def __call__(self, img):
        channel_mask = (torch.rand(img.shape) > self.drop_prob).float()
        noisy_img = img * channel_mask
        return noisy_img

### Step 2: Creating a Dataset of Paired Images: Original and Noisy
We define a dataset that pairs each original image with a corresponding noisy image. This dataset is structured so that each data sample includes both the clean image (as the label) and the noisified version (as the feature). We then create PyTorch dataloaders where each element contains a pair of original and noisy images.

In [ ]:
# PairDataset class that returns a pair of images: (original, noisy)
class PairDataset(Dataset):
    def __init__(self, dataset_origin, dataset_noisy):
        assert len(dataset_origin) == len(dataset_noisy), "Datasets must be of the same length"
        self.dataset_origin = dataset_origin
        self.dataset_noisy = dataset_noisy

    def __getitem__(self, index):
        clean_img, _ = self.dataset_origin[index]
        noisy_img, _ = self.dataset_noisy[index]
        return clean_img, noisy_img

    def __len__(self):
        return len(self.dataset_origin)

In [ ]:
# Function to create paired dataset for specific classes
def create_paired_dataloader(classes, num_samples=1500, batch_size=64, drop_prob=0.5, split='train'):
    """
    Creates a DataLoader for a paired dataset with noisy and clean images from specified classes.

    Parameters:
    - classes (list): List of class labels to include (e.g., [0, 1, 2, 3, 4] or [5, 6, 7, 8, 9]).
    - num_samples (int): Number of samples to use from the filtered classes.
    - batch_size (int): Batch size for the DataLoader.
    - drop_prob (float): Drop probability for AddNoiseTransform.
    - split (str): Dataset split to load ('train' or 'test').

    Returns:
    - DataLoader: A DataLoader for the paired dataset.
    """

    # Define transformations
    transform_clean = transforms.ToTensor()
    transform_noisy = transforms.Compose([
        transforms.ToTensor(),
        AddNoiseTransform(drop_prob=drop_prob)
    ])

    # Load the specified dataset split
    dataset_clean_full = datasets.SVHN(root='./data', split=split, download=True, transform=transform_clean)
    dataset_noisy_full = datasets.SVHN(root='./data', split=split, download=True, transform=transform_noisy)

    # Filter to include only samples with specified classes
    indices = [i for i, (_, label) in enumerate(dataset_clean_full) if label in classes]

    # Take only the first `num_samples` samples after filtering
    subset_indices = indices[:num_samples]

    # Create clean and noisy subsets
    dataset_clean = Subset(dataset_clean_full, subset_indices)
    dataset_noisy = Subset(dataset_noisy_full, subset_indices)

    # Create the paired dataset
    paired_dataset = PairDataset(dataset_clean, dataset_noisy)

    # DataLoader for paired dataset
    paired_dataloader = DataLoader(paired_dataset, batch_size=batch_size, shuffle=True)

    return paired_dataloader

## **Task A (10 points): Implementing a CNN for Image Denoising**

In this task, you will implement a Convolutional Neural Network (CNN) to map noisy images to their original, clean versions. This architecture consists of two convolutional layers.

### **Architecture Details**

1. **First Convolutional Layer**
   - **Setup**: `kernel_size=3`, `padding=1`, `stride=1` to maintain image dimensions.
   - **Input**: 32x32x3 images.
   - **Output**: Feature maps of size 32x32x30.
   - **Activation**: Apply ReLU for non-linearity.

2. **Second Convolutional Layer**
   - **Setup**: Same kernel size, padding, and stride as above.
   - **Output**: 32x32x3 to match the original image dimensions.
   - **Activation**: Sigmoid to constrain values between 0 and 1.

3. **Hyperparameters**
   - `input_channels=3`, `output_channels=3`, `feature_maps=30`.

### **Implementation Requirements**

Using these guidelines, implement the CNN model in PyTorch with `torch.nn.Conv2d` for the layers, and `torch.nn.ReLU` and `torch.nn.Sigmoid` for activations.

In [ ]:
class ImageDenoisingCNN(nn.Module):
    def __init__(self):
        super(ImageDenoisingCNN, self).__init__()


        ########################
        ########################
        #### YOUR CODE HERE ####
        ########################
        ########################

    def forward(self, x):

        ########################
        ########################
        #### YOUR CODE HERE ####
        ########################
        ########################


To understand the model’s complexity, we can calculate the total number of trainable parameters in ImageDenoisingCNN. Each parameter contributes to the model's ability to learn patterns in the data.

In [ ]:
model = ImageDenoisingCNN()

# Calculate and print the number of trainable parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Total number of trainable parameters in ImageDenoisingCNN:', num_params)

## **Task B (20 points)**: Denoising SVHN Images
This task involves building and evaluating a model that can remove noise from SVHN images in the first 5 classes (0 through 4). The model is trained and tested on a dataset of noisy images paired with their clean counterparts, using only 1,500 samples per class.

**Objectives**:
1. Complete the training function: This function should return the average training and test losses for each epoch.
2. Complete the testing function: This function should compute the average test loss across all batches.
3. Denoise SVHN Images: Train the model to denoise images from the first 5 classes.

Include the last epoch train/test values and last epoch sample images in your report.

In [ ]:
# Display a single row of images with specified columns
def show_images_grid2(images, title, cols):
    fig, axes = plt.subplots(1, cols, figsize=(cols * 2, 2))
    if cols == 1:
        axes = [axes]

    for idx in range(cols):
        if idx < len(images):
            img = images[idx].numpy().transpose((1, 2, 0))
            axes[idx].imshow(img)
            axes[idx].set_title(title)
            axes[idx].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
def train(train_loader, test_loader, model, epochs, loss_function, optimizer, device='cuda'):
    """
    Train the model on the training dataset and evaluate it on the test dataset.

    Parameters:
    - train_loader: DataLoader for the training set
    - test_loader: DataLoader for the test set
    - model: Neural network model to train
    - epochs: Number of training epochs
    - loss_function: Loss function for training
    - optimizer: Optimizer for updating model weights
    - device: Device to use for training ('cuda' or 'cpu')

    Returns:
    - train_loss_epochs: List of average training losses per epoch
    - test_loss_epochs: List of average test losses per epoch
    """

    # Move model to the specified device

    ########################
    ########################
    #### YOUR CODE HERE ####
    ########################
    ########################

    train_loss_epochs = []
    test_loss_epochs = []

    # Loop over the dataset for a specified number of epochs
    for epoch in range(epochs):
        model.train()  # Set model to training mode
        train_loss_batches = []

        # Loop over batches in the training data

        ########################
        ########################
        #### YOUR CODE HERE ####
        ########################
        ########################

            # Display sample results every 5 epochs, at the last batch of each epoch
            if epoch % 5 == 0 and batch_idx == len(train_loader) - 1:
                show_images_grid2(clean_images[:5].detach().cpu(), title= "Clean", cols=5)
                show_images_grid2(noisy_images[:5].detach().cpu(), title= "Noisy", cols=5)
                show_images_grid2(denoised_images[:5].detach().cpu(), title= "Denoised", cols=5)


        # Calculate average training loss for the epoch

    ########################
    ########################
    #### YOUR CODE HERE ####
    ########################
    ########################

        # Evaluate model on the test set and calculate test loss

    ########################
    ########################
    #### YOUR CODE HERE ####
    ########################
    ########################

        # Print losses for the current epoch
        print(f'Epoch {epoch+1:02d}/{epochs:02d} - Train Loss: {train_loss_epoch:.6f}, Test Loss: {test_loss_epoch:.6f}')

    return train_loss_epochs, test_loss_epochs

In [ ]:
def evaluate(dataloader, model, loss_function, device='cuda'):
    """
    Evaluate the model on the test dataset and return the average loss.
    """

    ########################
    ########################
    #### YOUR CODE HERE ####
    ########################
    ########################

    with torch.no_grad():  # Disable gradient calculation for testing

    ########################
    ########################
    #### YOUR CODE HERE ####
    ########################
    ########################

    return np.mean(test_losses)  # Return average loss over the dataset

In [ ]:
learning_rate = 0.05
batch_size = 64
drop_rate = 0.3
num_samples = 1500
num_epochs = 100



########################
########################
#### YOUR CODE HERE ####
########################
########################


# Load data

########################
########################
#### YOUR CODE HERE ####
########################
########################


# Train the model
train_loss_epochs, test_loss_epochs = train(paired_trainloader_first_five, paired_testloader_first_five,\
                                            model, num_epochs, criterion, optimizer, device)

## **Task C (5 points): Plotting Training and Testing Losses Over Epochs**

To visualize the learning process of your model, plot the training and testing losses over each epoch. This allows you to evaluate model performance over time and helps identify potential issues, such as overfitting or underfitting.

Include this plot in your report.


In [ ]:
    ########################
    ########################
    #### YOUR CODE HERE ####
    ########################
    ########################

## **Task D (10 points): Denoising Last 5 Classes, Reporting Loss, and Visualization**

Use your model to denoise images from classes 5-9 of the SVHN dataset. Report the test loss and visualize clean, noisy, and denoised images side by side. Include these results in your report.


In [ ]:
# Load test data for classes 5-9 with 1,500 samples per class

    ########################
    ########################
    #### YOUR CODE HERE ####
    ########################
    ########################


print(f'Number of batches in paired_testloader_last_five: {len(paired_testloader_last_five)}')



# Evaluate model on the last 5 classes test set and report loss

    ########################
    ########################
    #### YOUR CODE HERE ####
    ########################
    ########################

In [ ]:
# Visualize clean, noisy, and denoised images on last 5 classes
    ########################
    ########################
    #### YOUR CODE HERE ####
    ########################
    ########################

# Display clean, noisy, and denoised images
show_images_grid2(clean_images[:5].detach(), title="Clean", cols=5)
show_images_grid2(noisy_images[:5].detach(), title="Noisy", cols=5)
show_images_grid2(denoised_images, title="Denoised", cols=5)

# <font color="red"> Task 3: Kaggle Competition (25 + 20 Bonus Points) </font>

For this task, you will participate in a Kaggle competition. Follow the steps below to maximize your performance and gain additional points:

1. **Account Setup**: Create a Kaggle account if you don’t already have one, and log in. Make sure to include your Kaggle username in both your report and code submission for verification purposes.

2. **Competition Details**: Access the competition at the link provided below and thoroughly read the description and requirements. Implement your solution in a separate file (not in this notebook) and aim to achieve the highest score possible on the leaderboard.

   - **[Kaggle Competition Link](<https://www.kaggle.com/t/37714339d17855022339af848ed11e96>)**

3. **Submission on A2L**: Ensure your A2L submission includes:
   - **Final Code**: Upload your final code file used for the Kaggle competition.
   - **Report** (`kaggle.pdf`): Provide a detailed report explaining your model and approach, along with the code used in your last Kaggle submission.
     - Start the report with your final accuracy score.
     - Clearly mention your Kaggle username for verification.
   - **Submission File** (`submission.csv`): Upload the final `submission.csv` file used in the competition as a record of your performance.

4. **Bonus Points**: Additional points will be awarded to the top-performing students on the competition leaderboard.
